In [17]:
from gammapy.datasets import Datasets
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ConstantSpatialModel,
    SkyModel,
)
from gammapy.datasets.map import MapEvaluator
from gammapy.maps import Map
from gammapy.makers.utils import _map_spectrum_weight
import numpy as np

In [18]:
def compute_reco_exposure(dataset, spectral_model=None):
    """
    Create and exposure map in reco energies
    Parameters
    ----------
    dataset:`~gammapy.cube.MapDataset` or `~gammapy.cube.MapDatasetOnOff`
            the input dataset
    spectral_model: `~gammapy.modeling.models.SpectralModel`
            assumed spectral shape. If none, a Power Law of index 2 is assumed
    """
    if spectral_model is None:
        spectral_model = PowerLawSpectralModel()
    model = SkyModel(
        spatial_model=ConstantSpatialModel(), spectral_model=spectral_model
    )
    kernel = None
    if dataset.edisp is not None:
        kernel = dataset.edisp.get_edisp_kernel(position=dataset._geom.center_skydir)
    meval = MapEvaluator(model=model, exposure=dataset.exposure, edisp=kernel)
    npred = meval.compute_npred()
    e_reco = dataset._geom.get_axis_by_name("energy").edges
    ref_flux = spectral_model.integral(e_reco[:-1], e_reco[1:])
    reco_exposure = npred / ref_flux[:, np.newaxis, np.newaxis]
    return reco_exposure


In [19]:
datasets = Datasets.read(
        "$GAMMAPY_DATA/fermi-3fhl-crab",
        filedata="Fermi-LAT-3FHL_datasets.yaml",
        filemodel="Fermi-LAT-3FHL_models.yaml",
    )
dataset = datasets[0]
dataset.psf = None
dataset.edisp = None
etrue = dataset._geom.get_axis_by_name("energy").copy(name="energy_true")
geom_etrue = dataset._geom.to_image().to_cube([etrue])
dataset.exposure = Map.from_geom(
    geom=geom_etrue, data=dataset.exposure.data, unit=dataset.exposure.unit
)

/opt/anaconda3/envs/gammapy-dev/lib/python3.7/site-packages/astropy/wcs/wcs.py:687: FITSFixedWarning: 'datfix' made the change 'Set DATE-REF to '1858-11-17' from MJD-REF'.
  FITSFixedWarning)
/opt/anaconda3/envs/gammapy-dev/lib/python3.7/site-packages/astropy/wcs/wcs.py:687: FITSFixedWarning: 'datfix' made the change 'Set DATE-REF to '1858-11-17' from MJD-REF'.
  FITSFixedWarning)


In [20]:
exposure = dataset.exposure
exposure_reco = compute_reco_exposure(dataset)
exposure_wt = _map_spectrum_weight(dataset.exposure)

In [21]:
print("original exposure: ",exposure.quantity.sum())
print("exposure in ereco: ",exposure_reco.quantity.sum())
print("exposure weighted: ",exposure_wt.quantity.sum())

original exposure:  2916306127242266.5 cm2 s
exposure in ereco:  8881770290.89526 cm2 s
exposure weighted:  596692266187102.2 cm2 s
